<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem III
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Apache Spark;</li>
  <li>Data Wrangling com Spark.</li>
</ol>

---

# **Exercícios**

## 1\. Apache Spark

Replique as atividades do item 2.1 e 2.2 para instalar e configurar um cluster Apache Spark na máquina virtual do Google Colab.

In [12]:
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz



In [13]:
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [14]:
!pip install -q pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 15.9 MB/s eta 0:00:00


## 2\. Data Wrangling

A base de dados presente neste [link](https://www.kaggle.com/datasets/bank-of-england/a-millennium-of-macroeconomic-data) contem dados macroeconômicos sobre o Reino Unido desde o século 13.

**2.1\. Data**

Faça o download dos dados utilizando a máquina virutal do Google Colab com o código abaixo.

In [17]:
!wget -q "https://raw.githubusercontent.com/cluster-apps-on-docker/spark-standalone-cluster-on-docker/master/build/workspace/data/uk-macroeconomic-data.csv" -O "uk-macroeconomic-data.csv"

**2.2. Wrangling**

Processe os dados para que a base de dados final apresente os valores da taxa de desemprego (`Unemployment rate`) e população (`Population (GB+NI)`) estejam ordenados por ano decrescente:

```csv
year,population,unemployment_rate
...,...,...
```

Para isso, utilize:

 - Pandas

In [18]:
import pandas as pd

dados_macroeconomicos_UK = pd.read_csv('uk-macroeconomic-data.csv')
dados_macroeconomicos_UK.head(3)

,Description,Real GDP of England at market prices,Real GDP of England at factor cost,"Real UK GDP at market prices, geographically-consistent estimate based on post-1922 borders","Real UK GDP at factor cost, geographically-consistent estimate based on post-1922 borders","Index of real UK GDP at factor cost - based on changing political boundaries,",Composite estimate of English and (geographically-consistent) UK real GDP at factor cost,HP-filter of log of real composite estimate of English and UK real GDP at factor cost,"Real UK gross disposable national income at market prices, constant border estimate",Real consumption,...,UK Public sector debt.1,UK Public sector debt.2,Central Government Gross Debt,Central Government Gross Debt.1,Trade deficit,Trade deficit.1,Current account,Current account .1,Current account deficit including estimated non-monetary bullion flows,Current account deficit including estimated non-monetary bullion flows.1
0,Units,"£mn, Chained Volume measure, 2013 prices","£mn, Chained Volume measure, 2013 prices","£mn, Chained Volume measure, 2013 prices","£mn, Chained Volume measure, 2013 prices","GB before 1801, GB+Ireland 1801-1920, GB + Nor...",2013=100,approx. % difference from trend,"£mn, Chained Volume measure, 2013 prices. Nom...","£mn, Chained Volume measure, 2013 prices",...,as a % of nominal GDP: measure 1,as a % of nominal GDP: measure 2,"Financial year end, nominal par value £mn","Financial year end, market value £mn",£mn,as a % of nominal GDP,£mn,as a % of nominal GDP,£mn,as a % of nominal GDP
1,1209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
dados_macroeconomicos_UK = dados_macroeconomicos_UK.loc[1:]
colunas_interesse = ['Description', 'Population (GB+NI)', 'Unemployment rate']
dados_macroeconomicos_UK = dados_macroeconomicos_UK[colunas_interesse].dropna().sort_values('Description', ascending=False)
dados_macroeconomicos_UK

,Description,Population (GB+NI),Unemployment rate
808,2016,65573,4.90
807,2015,65110,5.38
806,2014,64597,6.18
805,2013,64106,7.61
804,2012,63705,7.97
...,...,...,...
651,1859,24138,3.27
650,1858,23914,5.23
649,1857,23689,3.95
648,1856,23466,3.52


In [20]:
dados_macroeconomicos_UK.columns = ['Year', 'Population', 'Unemployment rate']
dados_macroeconomicos_UK.head(3)

,Year,Population,Unemployment rate
808,2016,65573,4.90
807,2015,65110,5.38
806,2014,64597,6.18


In [21]:
dados_macroeconomicos_UK.to_csv('uk-macroeconomic-data-cleaned.csv', index=False)

 - PySpark

In [22]:
!pip install pyspark
#import spark session
from pyspark.sql import sparksession
#create a spark session
spark=sparksession.builder.appname('readcsv').getorcreate()

ImportError: cannot import name 'sparksession' from 'pyspark.sql' (/usr/local/lib/python3.10/dist-packages/pyspark/sql/__init__.py)

In [23]:
dados_macroeconomicos_UK = spark.read.csv(
    path='uk-macroeconomic-data.csv',
    sep=',',
    header=True
)

dados_macroeconomicos_UK.show(n=4)

NameError: name 'spark' is not defined

In [24]:
dados_macroeconomicos_UK.printSchema()

AttributeError: 'DataFrame' object has no attribute 'printSchema'